<a href="https://colab.research.google.com/github/JackelinneSC/Data-Structure-II/blob/master/sentiment_analysis_bts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**BTS Sentiment Analysis 💜**

Sentiment analisys using comments from YT videos related to BTS in Jan 2021.
The dataset used is extracted from Kaggle:
https://www.kaggle.com/datasets/seungguini/bts-youtube-comments?select=bts_2021_1.csv


Unzip dataset from Google Drive. Run this cell one time.

In [ ]:
# !unzip "/content/drive/MyDrive/BTS YT Comment/bts_2021_1.csv.zip" -d "/content/drive/MyDrive/BTS YT Comment/"

Archive:  /content/drive/MyDrive/BTS YT Comment/bts_2021_1.csv.zip
  inflating: /content/drive/MyDrive/BTS YT Comment/bts_2021_1.csv  


Check for GPU availability

In [ ]:

#print("GPU", "available!" if tf.config.list_physical_devices("GPU")else "not available :(")

Access our data and importing pandas.

In [ ]:
# Checkoyt the labels of our data
import pandas as pd
bts_df = pd.read_csv("/content/drive/MyDrive/BTS YT Comment/bts_2021_1.csv")
bts_df.head()

,query,url,title,upload_date,channel,views,likes,dislikes,comment_count,comment_text,comment_author,comment_date,comment_likes
0,bts,https://www.youtube.com/watch?v=S8GpX3SAeig,5 Hour BTS Piano Playlist | Study & Relax with...,2021-01-01T10:58:00Z,DooPiano,2444982,119269,501,3224,♪ Listen on Spotify!: https://spoti.fi/3gC9GfA...,DooPiano,2021-01-01T10:58:20Z,3884
1,bts,https://www.youtube.com/watch?v=S8GpX3SAeig,5 Hour BTS Piano Playlist | Study & Relax with...,2021-01-01T10:58:00Z,DooPiano,2444982,119269,501,3224,My ears: *relaxing* My hands: *writing* My le...,•ɱĭss süğą•,2021-01-29T15:42:43Z,4077
2,bts,https://www.youtube.com/watch?v=S8GpX3SAeig,5 Hour BTS Piano Playlist | Study & Relax with...,2021-01-01T10:58:00Z,DooPiano,2444982,119269,501,3224,Parents: You have to make us proud Partner: Y...,Leian Xyrielle Dayahan,2021-03-09T00:52:35Z,827
3,bts,https://www.youtube.com/watch?v=S8GpX3SAeig,5 Hour BTS Piano Playlist | Study & Relax with...,2021-01-01T10:58:00Z,DooPiano,2444982,119269,501,3224,Little boy: “Are you an angel?” Girl: “What?”...,Lisha,2021-02-12T15:48:08Z,921
4,bts,https://www.youtube.com/watch?v=S8GpX3SAeig,5 Hour BTS Piano Playlist | Study & Relax with...,2021-01-01T10:58:00Z,DooPiano,2444982,119269,501,3224,Reasons to live: “Suicide doesn’t stop the ...,Grace Cho,2021-02-02T18:39:00Z,2248


In [ ]:
import spacy 
import nltk
nlp = spacy.load('en', disable=['parser', 'ner'])

**Tokenization**
Tokenization is the action of breaking every sentence into indiviaul words.

In [ ]:
# Reading the second comment from the dataset
comment_text = bts_df.iloc[1]["comment_text"]
comment_text

"My ears: *relaxing* My hands: *writing*  My legs: *dancing* My mouth: *singin'*"

Perfoming tokenization with NLTK.


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk.tokenize import word_tokenize

print(word_tokenize(comment_text))

['My', 'ears', ':', '*relaxing*', 'My', 'hands', ':', '*writing*', 'My', 'legs', ':', '*dancing*', 'My', 'mouth', ':', "*singin'*"]


Perfoming tokenization with Sparcy.


In [ ]:
print([str(token) for token in nlp(comment_text) if not token.is_punct])

['My', 'ears', 'relaxing', 'My', 'hands', 'writing', ' ', 'My', 'legs', 'dancing', 'My', 'mouth', 'singin']


Next step is to remove useless information. For example, symbols, digits, punctuations. I will use spacy combined with regex.

In [ ]:
import re

# Tokenization and removal of punctuation marks
words = [str(token) for token in nlp(comment_text) if not token.is_punct]

# Remove digits and other symbols except "@" -- used to remove email
words = [re.sub(r"[^A-Za-z@]", "", word) for word in words]

# Remove websites and email address
words = [re.sub(r"\S+com", "", word) for word in words]
words = [re.sub(r"S+@\S", "", word) for word in words]

# Remove empty spaces
words = [word for word in words if word !=' ']

words

['My',
 'ears',
 'relaxing',
 'My',
 'hands',
 'writing',
 '',
 'My',
 'legs',
 'dancing',
 'My',
 'mouth',
 'singin']

**Stopwords**


Next step in preprocessing the data is to remove useless words, for example, "and", "we","I", "My", "a", "of" etc. 

Stopwords are words that frequently appear in written documents but don't have a significant meanings.

We can import the stopwords from the NLTK library.

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# importing list of stopwords
stopwords = nltk.corpus.stopwords.words("english")

# lowercase stopwords
stopwords = [s.lower() for s in stopwords]

# exclude the stopwords from the comments
words = [word.lower() for word in words if word.lower() not in stopwords]

# view comment without stopwords
words

['ears',
 'relaxing',
 'hands',
 'writing',
 '',
 'legs',
 'dancing',
 'mouth',
 'singin']

**Lemmatization**

This step is about converting words to its original form.

This transformation is implemented with Spacy. After lemmatization, each comment will transform into a list of words that are all in their original forms.

In [ ]:
words_a = [token.lemma_ for token in nlp(comment_text) if not token.is_punct]
words_a

['-PRON-',
 'ear',
 'relax',
 '-PRON-',
 'hand',
 'writing',
 ' ',
 '-PRON-',
 'leg',
 'dance',
 '-PRON-',
 'mouth',
 'singin']

**Preprocess data function**

Wrapping up all the steps and creating a magic function that performs all the data pre-processing.


The "title" and "comment_text" are the columns that I will apply text preprocessing procedures on.

In [ ]:
comment_df = bts_df[["title","comment_text"]]
comment_df

,title,comment_text
0,5 Hour BTS Piano Playlist | Study & Relax with...,♪ Listen on Spotify!: https://spoti.fi/3gC9GfA...
1,5 Hour BTS Piano Playlist | Study & Relax with...,My ears: *relaxing* My hands: *writing* My le...
2,5 Hour BTS Piano Playlist | Study & Relax with...,Parents: You have to make us proud Partner: Y...
3,5 Hour BTS Piano Playlist | Study & Relax with...,Little boy: “Are you an angel?” Girl: “What?”...
4,5 Hour BTS Piano Playlist | Study & Relax with...,Reasons to live: “Suicide doesn’t stop the ...
...,...,...
28286,BTS OF BHAI KI JOURNEY | INTRODUCTION (PART 1)...,Kdkk
28287,BTS OF BHAI KI JOURNEY | INTRODUCTION (PART 1)...,Dusra part kadhi yetoy 🙄
28288,BTS OF BHAI KI JOURNEY | INTRODUCTION (PART 1)...,😁😁😂😂😂
28289,BTS OF BHAI KI JOURNEY | INTRODUCTION (PART 1)...,Kay tha bhai ye 🤔🤔🤔 but achhha tha 👍🐱🐱🐱😁


In [ ]:
def text_preprocessing(input):
  # Applying tokenization, removal of punctuation and lemmatization
  words = [token.lemma_ for token in nlp(input) if not token.is_punct]

  # Remove digits and other symbols except "@" -- used to remove email
  words = [re.sub(r"[^A-Za-z@]", "", word) for word in words]

  # Remove websites and email address
  words = [re.sub(r"\S+com", "", word) for word in words]
  words = [re.sub(r"S+@\S", "", word) for word in words]

  # Remove empty spaces
  words = [word for word in words if word !=' ']    
  words = [word for word in words if len(word)!=0]
